In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from .Value import Value
from .Neuron import Neuron

In [ ]:
from graphviz import Digraph

def trace(root):
	# builds a set of all nodes and edges in a graph
	nodes, edges = set(), set()
	def build(v):
		if v not in nodes:
			nodes.add(v)
			for child in v._prev:
				edges.add((child, v))
				build(child)
	build(root)
	return nodes, edges

def draw_dot(root):
	dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR: Left to Right
	nodes, edges = trace(root)
	for n in nodes:
		uid = str(id(n))
		# for any value in the graph, create a rectangular ('record') node for it
		dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label ,n.data, n.grad), shape='record')
		if n._op:
			# if this value is a result of some operation, then create an op node for it 
			dot.node(name = uid + n._op, label = n._op)
			# and connect this node to it
			dot.edge(uid + n._op, uid)

	for n1, n2 in edges:
		# connect n1 to the op of n2
		dot.edge(str(id(n1)), str(id(n2)) + n2._op)

	return dot


In [ ]:
def topo(root):
	topo = []
	visited = set()
	def build_topo(v):
		if v not in visited:
			visited.add(v)
			for child in v._prev:
				build_topo(child)
			topo.append(v)
	build_topo(root)
	return topo


In [ ]:
def tweak(data, iterations=20, inc=0.05):
	xs, ys, n = data
	for k in range(iterations):
		# Forward Pass
		ypred = [n(x) for x in xs]
		loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

		# Zero grad
		for p in n.parameters():
			p.grad = 0.0

		# Backward
		loss.backward()

		# Update
		for p in n.parameters():
			p.data -= inc * p.grad

		print(k, loss.data)
	return (xs, ys, n)
